###  Vector stores and retrievers
This video tutorial will familiarize you with LangChain's vector store and retriever abstractions. These
abstractions are designed to support retrieval of data-- from (vector) databases and other sources-- for
integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part
of model inference, as in the case of retrieval-augmented generation.
We will cover

•Documents

•Vector stores

•Retrievers



### Documents
LangChain implements a Document abstraction, which is intended to represent a unit of text and
associated metadata. It has two attributes:

- page content: a string representing the content;
- metadata: a dict containing arbitrary metadata.
The metadata attribute can capture information about the source of the document, its relationship
to other documents, and other information. Note that an individual Document object often
represents a chunk of a larger document.

Let's generate some sample documents:

In [1]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent animals, often appreciated for their playful nature.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Birds can be colorful and are often kept as pets for their singing abilities.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Fish are popular pets, known for their calming presence in aquariums.",
        metadata={"source": "aquatic-pets-doc"},
    ),
    Document(
        page_content="Reptiles, such as turtles and snakes, are unique pets that require special care.",
        metadata={"source": "reptile-pets-doc"},
    ),
]
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent animals, often appreciated for their playful nature.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Birds can be colorful and are often kept as pets for their singing abilities.'),
 Document(metadata={'source': 'aquatic-pets-doc'}, page_content='Fish are popular pets, known for their calming presence in aquariums.'),
 Document(metadata={'source': 'reptile-pets-doc'}, page_content='Reptiles, such as turtles and snakes, are unique pets that require special care.')]

In [3]:
import os 
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
llm = ChatGroq(model="Llama3-8b-8192",groq_api_key = groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002207AB32C50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002207AB31AE0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

e:\GenAi\Langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
## VectorStores
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents,embedding = embeddings)       # documents are embedding using this embedding technique and store in chroma db
vectorstore

e:\GenAi\Langchain\venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [8]:
vectorstore.similarity_search("cats")

[Document(id='2b611930-2c14-4fac-88ed-527d4305b6a2', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent animals, often appreciated for their playful nature.'),
 Document(id='4d14795b-f0e0-43e4-8a61-613089e6e956', metadata={'source': 'aquatic-pets-doc'}, page_content='Fish are popular pets, known for their calming presence in aquariums.'),
 Document(id='e52abe38-18d7-4b7c-88c3-f6931f5e5aaf', metadata={'source': 'reptile-pets-doc'}, page_content='Reptiles, such as turtles and snakes, are unique pets that require special care.'),
 Document(id='bb02885b-71a0-4883-a06e-a145073228da', metadata={'source': 'bird-pets-doc'}, page_content='Birds can be colorful and are often kept as pets for their singing abilities.')]

In [9]:
#  ASync query

await vectorstore.asimilarity_search("cats")

[Document(id='2b611930-2c14-4fac-88ed-527d4305b6a2', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent animals, often appreciated for their playful nature.'),
 Document(id='4d14795b-f0e0-43e4-8a61-613089e6e956', metadata={'source': 'aquatic-pets-doc'}, page_content='Fish are popular pets, known for their calming presence in aquariums.'),
 Document(id='e52abe38-18d7-4b7c-88c3-f6931f5e5aaf', metadata={'source': 'reptile-pets-doc'}, page_content='Reptiles, such as turtles and snakes, are unique pets that require special care.'),
 Document(id='bb02885b-71a0-4883-a06e-a145073228da', metadata={'source': 'bird-pets-doc'}, page_content='Birds can be colorful and are often kept as pets for their singing abilities.')]

In [10]:
vectorstore.similarity_search_with_score("cats")

e:\GenAi\Langchain\venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[(Document(id='2b611930-2c14-4fac-88ed-527d4305b6a2', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent animals, often appreciated for their playful nature.'),
  0.8897172212600708),
 (Document(id='4d14795b-f0e0-43e4-8a61-613089e6e956', metadata={'source': 'aquatic-pets-doc'}, page_content='Fish are popular pets, known for their calming presence in aquariums.'),
  1.3621560335159302),
 (Document(id='e52abe38-18d7-4b7c-88c3-f6931f5e5aaf', metadata={'source': 'reptile-pets-doc'}, page_content='Reptiles, such as turtles and snakes, are unique pets that require special care.'),
  1.3881986141204834),
 (Document(id='bb02885b-71a0-4883-a06e-a145073228da', metadata={'source': 'bird-pets-doc'}, page_content='Birds can be colorful and are often kept as pets for their singing abilities.'),
  1.4091432094573975)]

## Retrievers
LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into
LangChain Expression language chains.

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and
asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we
wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the
similarity_search method:

In [ ]:
# Retrievers

from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

e:\GenAi\Langchain\venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[[Document(id='2b611930-2c14-4fac-88ed-527d4305b6a2', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent animals, often appreciated for their playful nature.')],
 [Document(id='6ccb7563-bb2a-4f9c-b973-c98884af72e0', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

Vectorstores implement an as_retriever method that will generate a Retriever, specifically a
VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify
what methods of the underlying vector store to call, and how to parameterize them. For instance, we can
replicate the above with the following:

In [13]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}               # batch thing
)

retriever.batch(["cat","dog"])

e:\GenAi\Langchain\venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[[Document(id='2b611930-2c14-4fac-88ed-527d4305b6a2', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent animals, often appreciated for their playful nature.')],
 [Document(id='6ccb7563-bb2a-4f9c-b973-c98884af72e0', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [ ]:
# Rag = Retriever Autmented Generation

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
    Answer the question using provided context only    
    {question}

    context:
    {context}
"""

prompt = ChatPromptTemplate.from_messages([("human",message)])

rag_chain = {"context":retriever,"question":RunnablePassthrough()} | prompt | llm

response = rag_chain.invoke("tell me about dog")

response

e:\GenAi\Langchain\venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


AIMessage(content='According to the provided context, dogs are great companions, known for their loyalty and friendliness.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 86, 'total_tokens': 106, 'completion_time': 0.024653392, 'prompt_time': 0.027222823, 'queue_time': 0.325011137, 'total_time': 0.051876215}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--5d365a9a-c6ee-4ef2-97d7-5d96c02f6ff6-0', usage_metadata={'input_tokens': 86, 'output_tokens': 20, 'total_tokens': 106})